https://leetcode.com/articles/a-recursive-approach-to-segment-trees-range-sum-queries-lazy-propagation/
を参考にする

- Treeを配列に置き換える(Heapと同じ)
- Leafが元の配列の一つ一つの要素
- Leafは左詰にして、2のn乗に満たない場合はNoneを収納する(Perfect Binary Treeを作る)

- あるNodeのidxについて、Parentは(idx-1)//2、LeftChildはidx*2+1, RightChildはidx*2+2
- Nodeのidxと、そのNodeの表す区間のidxは全然関係ない

## Build Segment Tree

ある区間の最小値を求めるやつ

In [119]:
# class Node:
#     def __init__(self, val=None, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right

In [120]:
# memo: 区間のmidの"切り捨て"がleft childの区間の上端にする→詰まってるNodeだけ見ると、complete binary treeになる
def build_segment_tree(segment_tree, node_idx, range_left, range_right, nums):
    if range_left == range_right:
        segment_tree[node_idx] = arr[range_left]
        return segment_tree[node_idx]
        
    segment_tree[node_idx] = build_segment_tree(
        segment_tree, node_idx * 2 + 1, range_left, (range_left + range_right) // 2, nums
    ) + build_segment_tree(
        segment_tree, node_idx * 2 + 2, (range_left + range_right) // 2 + 1, range_right, nums
    )
    
    return segment_tree[node_idx]
    

In [121]:
nums = [1,2,3,4,5,6,7]

num_ele = 16 - 1# (16>)8(>=7)　* 2。Noneを詰めてperfect binary treeになる。
segment_tree = [None] * num_ele

build_segment_tree(segment_tree, 0, 0, len(nums)-1, nums)

NameError: name 'arr' is not defined

In [122]:
segment_tree

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Build Segment Treeをクラスのメソッドにする

In [123]:
class SegmentTree:
    def __init__(self, original_nums):
        self.original_nums = original_nums
        self.segment_tree_arr = [None] * len(nums) * 4 # This has to be large enough. See: https://stackoverflow.com/questions/28470692/how-is-the-memory-of-the-array-of-segment-tree-2-2-ceillogn-1
        self.build_segment_tree(0, 0, len(nums)-1)
        
    # Treeでnode_idxに当たるNode以下のSegmentTreeをself.segment_tree_arrに収納する。
    # 返り値は、node_idxに当たるNodeに収納されるvalue
    def build_segment_tree(self, node_idx, range_left, range_right):
        if range_left == range_right:
            self.segment_tree_arr[node_idx] = self.original_nums[range_left]
            return self.segment_tree_arr[node_idx]

        self.segment_tree_arr[node_idx] = self.build_segment_tree(
            node_idx * 2 + 1, range_left, (range_left + range_right) // 2       # 区間のmidの"切り捨て"がleft childの区間の上端にする→詰まってるNodeだけ見ると、complete binary treeになる
        ) + self.build_segment_tree(
            node_idx * 2 + 2, (range_left + range_right) // 2 + 1, range_right
        )
        return self.segment_tree_arr[node_idx]

In [124]:
sg = SegmentTree(nums)
print(sg.segment_tree_arr)

[28, 10, 18, 3, 7, 11, 7, 1, 2, 3, 4, 5, 6, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


## Readメソッドを生やす

指定された区間の最小値を求める

In [131]:
class SegmentTree:
    def __init__(self, original_nums):
        self.original_nums = original_nums
        self.len_nums = len(original_nums)
        self.segment_tree_arr = [None] * len(nums) * 4 # This has to be large enough. See: https://stackoverflow.com/questions/28470692/how-is-the-memory-of-the-array-of-segment-tree-2-2-ceillogn-1
        self.build_segment_tree(0, 0, len(nums)-1)
        
    # Treeでnode_idxに当たるNode以下のSegmentTreeをself.segment_tree_arrに収納する。
    # 返り値は、node_idxに当たるNodeに収納されるvalue
    def build_segment_tree(self, node_idx, range_left, range_right):
        if range_left == range_right:
            self.segment_tree_arr[node_idx] = self.original_nums[range_left]
            return self.segment_tree_arr[node_idx]

        self.segment_tree_arr[node_idx] = self.merge(
            self.build_segment_tree(
                node_idx * 2 + 1, range_left, (range_left + range_right) // 2       # 区間のmidの"切り捨て"がleft childの区間の上端にする→詰まってるNodeだけ見ると、complete binary treeになる
            ), self.build_segment_tree(
                node_idx * 2 + 2, (range_left + range_right) // 2 + 1, range_right
            )
        )
        return self.segment_tree_arr[node_idx]
    
    # find_minするやつ
    def read(self, left, right):
        range_left = max(left, 0)
        range_right = min(right, self.len_nums-1)
        if range_left > range_right:
            raise Exception
        return self._read(0, 0, self.len_nums-1, range_left, range_right)
       
    # Args: 
    #    idx: index of Node in the Segment Tree
    #   left, right: range where Node　equivalent to idx
    #   range_left, range_right: range of interest
    def _read(self, idx, left, right, range_left, range_right):
        if range_right < left:
            print(left, right, range_left, range_right)
            return None, 1
        if range_left > right:
            return None, 2 
        if (range_left == left) and (range_right == right):
            return self.segment_tree_arr[idx]
        
        mid = (left + right) // 2
        
        # search only in right child tree
        if mid + 1 <= range_left:
            return self._read(idx * 2 + 2, mid+1, right, mid+1, range_right)
        
        # search only in left child tree
        if range_right <= mid:
            return self._read(idx * 2 + 1, left, mid, range_left, mid)
        
        # else
        return self.merge(
            self._read(idx * 2 + 1, left,     mid,  range_left, mid),
            self._read(idx * 2 + 2, mid + 1, right, mid + 1,    range_right),                      
        )
        
    def merge(self, x, y):
        return min(x, y)

In [132]:
sg = SegmentTree(nums)
print(sg.segment_tree_arr)

[1, 1, 5, 1, 3, 5, 7, 1, 2, 3, 4, 5, 6, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [133]:
sg.read(1,5)

2

## Updateメソッドを生やす

指定されたindexに指定された値を設定する

In [134]:
class SegmentTree:
    def __init__(self, original_nums):
        self.original_nums = original_nums
        self.len_nums = len(original_nums)
        self.segment_tree_arr = [None] * len(nums) * 4 # This has to be large enough. See: https://stackoverflow.com/questions/28470692/how-is-the-memory-of-the-array-of-segment-tree-2-2-ceillogn-1
        self.build_segment_tree(0, 0, len(nums)-1)
        
    # Treeでnode_idxに当たるNode以下のSegmentTreeをself.segment_tree_arrに収納する。
    # 返り値は、node_idxに当たるNodeに収納されるvalue
    def build_segment_tree(self, node_idx, range_left, range_right):
        if range_left == range_right:
            self.segment_tree_arr[node_idx] = self.original_nums[range_left]
            return self.segment_tree_arr[node_idx]

        self.segment_tree_arr[node_idx] = self.merge(
            self.build_segment_tree(
                node_idx * 2 + 1, range_left, (range_left + range_right) // 2       # 区間のmidの"切り捨て"がleft childの区間の上端にする→詰まってるNodeだけ見ると、complete binary treeになる
            ), self.build_segment_tree(
                node_idx * 2 + 2, (range_left + range_right) // 2 + 1, range_right
            )
        )
        return self.segment_tree_arr[node_idx]
    
    # find_minするやつ
    def find(self, left, right):
        range_left = max(left, 0)
        range_right = min(right, self.len_nums-1)
        if range_left > range_right:
            raise Exception
        return self._find(0, 0, self.len_nums-1, range_left, range_right)
       
    # Args: 
    #    idx: index of Node in the Segment Tree
    #   left, right: range where Node　equivalent to idx
    #   range_left, range_right: range of interest
    def _find(self, idx, left, right, range_left, range_right):
        if range_right < left:
            print(left, right, range_left, range_right)
            return None, 1
        if range_left > right:
            return None, 2 
        if (range_left == left) and (range_right == right):
            return self.segment_tree_arr[idx]
        
        mid = (left + right) // 2
        
        # search only in right child tree
        if mid + 1 <= range_left:
            return self._find(idx * 2 + 2, mid+1, right, mid+1, range_right)
        
        # search only in left child tree
        if range_right <= mid:
            return self._find(idx * 2 + 1, left, mid, range_left, mid)
        
        # else
        return self.merge(
            self._find(idx * 2 + 1, left,     mid,  range_left, mid),
            self._find(idx * 2 + 2, mid + 1, right, mid + 1,    range_right),                      
        )
    
    def update(self, idx, val):
        if (idx < 0) or (self.len_nums-1 < idx):
            raise Exception
        self.idx_updated = idx
        self.val_updated = val
        self._update(0, 0, self.len_nums-1)
        return
        
    def _update(self, idx, left, right):
        if left == right: # reaching here means left == right == idx_updated
            self.segment_tree_arr[idx] = self.val_updated
            return
        mid = (left + right) // 2

        if idx <= mid:
            # go down to left child
            self._update(idx * 2 + 1, left, mid)
        else:
            # go down to right child
            self._update(idx * 2 + 2, mid+1, right)
        
        self.segment_tree_arr[idx] = self.merge(self.segment_tree_arr[idx*2+1], self.segment_tree_arr[idx*2+2])
        return 
        
    def merge(self, x, y):
        return min(x, y)

In [135]:
sg = SegmentTree(nums)
print(sg.segment_tree_arr)
sg.update(2,-1)
print(sg.segment_tree_arr)

[1, 1, 5, 1, 3, 5, 7, 1, 2, 3, 4, 5, 6, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[-1, -1, 5, -1, 3, 5, 7, 1, -1, 3, 4, 5, 6, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [137]:
sg.find(1,18)

-1

## Time Complexity

### build

time: O(n), space: O(4*n) [Depends on self.segment_tree_arr = [None] * len(nums) * 4]

### query

time: O(log n), space: O(1)

### update

time: O(log n), space: O(1)

## もう少し綺麗にかけた(Top-down)

あってるかの確認はしていない

In [ ]:
class SegmentTree:
    def __init__(self, original_nums):
        self.original_nums = original_nums
        self.len_nums = len(original_nums)
        self.segment_tree_arr = [None] * len(nums) * 4 # This has to be large enough. See: https://stackoverflow.com/questions/28470692/how-is-the-memory-of-the-array-of-segment-tree-2-2-ceillogn-1
        self.build_segment_tree(0, 0, len(nums)-1)
        
    # Treeでnode_idxに当たるNode以下のSegmentTreeをself.segment_tree_arrに収納する。
    # 返り値は、node_idxに当たるNodeに収納されるvalue
    def build_segment_tree(self, node_idx, range_left, range_right):
        if range_left == range_right:
            self.segment_tree_arr[node_idx] = self.original_nums[range_left]
            return self.segment_tree_arr[node_idx]

        self.segment_tree_arr[node_idx] = self.merge(
            self.build_segment_tree(
                node_idx * 2 + 1, range_left, (range_left + range_right) // 2       # 区間のmidの"切り捨て"がleft childの区間の上端にする→詰まってるNodeだけ見ると、complete binary treeになる
            ), self.build_segment_tree(
                node_idx * 2 + 2, (range_left + range_right) // 2 + 1, range_right
            )
        )
        return self.segment_tree_arr[node_idx]
        
        
    def update(self, index, val):
        self._update(index, 0, val, 0, self.len_nums-1)
        print(self.segment_tree_arr)
     
    # Update and returns merged value
    # Args:
    #   index: index in original array which should be updated
    #   index_in_seg_arr: index in segment_tree_arr
    #   left, right: range which node of idx_in_arr represents
    def _update(self, index, index_in_seg_arr, val, left, right):
        print(index, index_in_seg_arr, val, left, right, 'sef_arr: ', self.segment_tree_arr)
        
        if (index < left) or (right < index):
            return None
        if left == right:
            self.segment_tree_arr[index_in_seg_arr] = val
            return self.segment_tree_arr[index_in_seg_arr]
            
        mid = (left + right) // 2
        
        self.segment_tree_arr[index_in_seg_arr] = self.merge(
            self._update(index, index_in_seg_arr*2+1, val, left, mid),
            self._update(index, index_in_seg_arr*2+2, val, mid+1, right)
        )
        
        return self.segment_tree_arr[index_in_seg_arr]
        
    # Args:
    #   left, right: range which was given
    #   current_right, current_right: range where current node represents
    def sumRange(self, left, right):
        return self._sumRange(0, left, right, 0, self.len_nums-1)
        
    # Args:
    #   left, right: range which is our interest
    #   current_right, current_right: range where current node represents
    #   index_in_arr: index in self.segment_tree_arrreturn 
    def _sumRange(self, index_in_arr, left, right, current_left, current_right):
        if (left == current_left) and (right == current_right):
            return self.segment_tree_arr[index_in_arr]
        if (left > right):
            return None
        
        mid = (current_left + current_right) // 2
        return self.merge(
            self._sumRange(self, left, mid, current_left, mid),
            self._sumRange(self, mid+1, right, mid+1, current_right)
        )        
        
    def merge(self, x, y):
        if x is None:
            return y
        if y is None:
            return x
        return x + y
        

## Bottom-Up

In [1]:
class SegmentTree:
    def __init__(self, nums):
        self.len_nums = len(nums)
        self.segment_tree_arr = [None] * (len(nums)-1)  + nums
        
        for i in range(len(nums)-2, -1, -1):
            self.segment_tree_arr[i] = self.merge(
                self.segment_tree_arr[2*i+1], self.segment_tree_arr[2*i+2]
            )
            
    def update(self, index, val):
        crt_idx_in_seg = index + self.len_nums - 1
        self.segment_tree_arr[crt_idx_in_seg] = val
        
        crt_idx_in_seg = (crt_idx_in_seg-1) // 2
        while 0 <= crt_idx_in_seg:
            self.segment_tree_arr[crt_idx_in_seg] = self.segment_tree_arr[
                crt_idx_in_seg*2+1] + self.segment_tree_arr[crt_idx_in_seg*2+2]
            crt_idx_in_seg = (crt_idx_in_seg-1) // 2
       
    def sumRange(self, left, right):
        left = left + self.len_nums - 1
        right = right + self.len_nums - 1
        sum_range = 0
        
        while (left <= right):    ## left < rightにして最後に sum_range += self.segment_tree_arr[right]を加えるのはまずい。反例は見つけられてない。
            if left % 2 == 0:
                sum_range += self.segment_tree_arr[left]
                left += 1
            left = (left-1) // 2
            
            if right % 2 == 1:
                sum_range += self.segment_tree_arr[right]
                right -= 1
            right = (right - 1) // 2
         
        # sum_range += self.segment_tree_arr[right]
        return sum_range

    def merge(self, x, y):
        if x is None:
            return y
        if y is None:
            return x
        return x + y      
    